### **GOLD LAYER**

### **PRODUCT DIMENSION SCD TYPE-1 LOAD**

In [0]:
MERGE INTO retail_databricks_ws.gold.products_dim AS T
USING (
    SELECT DISTINCT
        productcode,
        productline,
        msrp
    FROM retail_databricks_ws.silver.silver_cleaned_tbl
    WHERE productcode is NOT NULL
) AS S
ON T.productcode = S.productcode
WHEN MATCHED THEN
  UPDATE SET
      T.productline = S.productline,
      T.msrp        = S.msrp
WHEN NOT MATCHED THEN
  INSERT (
      productcode,
      productline,
      msrp
  )
  VALUES (
      S.productcode,
      S.productline,
      S.msrp
  );

### **DATE DIMENSION LOAD**

In [0]:
MERGE INTO retail_databricks_ws.gold.date_dim AS T
USING (
    SELECT DISTINCT 
        CAST(date_format(orderdate,'yyyyMMdd') AS INT) as date_key,
        orderdate,
        month_id,
        qtr_id,
        year_id
    FROM retail_databricks_ws.silver.silver_cleaned_tbl
    WHERE orderdate IS NOT NULL
) AS S
ON T.date_key = S.date_key
WHEN NOT MATCHED THEN 
INSERT(date_key, orderdate, month_id, qtr_id, year_id)
VALUES(S.date_key, S.orderdate, S.month_id, S.qtr_id, S.year_id)

### **CUSTOMER DIMENSION SCD TYPE-2 LOAD**

In [0]:
MERGE INTO retail_databricks_ws.gold.customers_dim AS T
USING(
    SELECT DISTINCT 
        customername,
        phone,
        addressline1,
        addressline2,
        city,
        state,
        postalcode,
        country,
        territory,
        country_iso,
        dealsize,
        contactfirstname as firstname,
        contactlastname as lastname
    FROM (
        SELECT *,
               row_number() over(partition by customername order by silver_processed_timestamp DESC) as rn
               FROM retail_databricks_ws.silver.silver_cleaned_tbl
               WHERE customername IS NOT NULL
         )
WHERE rn = 1
) AS S
ON T.customername = S.customername AND
   T.is_current = TRUE
WHEN MATCHED AND (
    T.phone        IS DISTINCT FROM S.phone OR
    T.addressline1 IS DISTINCT FROM S.addressline1 OR
    T.addressline2 IS DISTINCT FROM S.addressline2 OR
    T.city         IS DISTINCT FROM S.city OR
    T.state        IS DISTINCT FROM S.state OR
    T.postalcode   IS DISTINCT FROM S.postalcode OR
    T.country      IS DISTINCT FROM S.country OR
    T.territory    IS DISTINCT FROM S.territory OR
    T.country_iso  IS DISTINCT FROM S.country_iso OR
    T.dealsize     IS DISTINCT FROM S.dealsize OR
    T.firstname    IS DISTINCT FROM S.firstname OR
    T.lastname     IS DISTINCT FROM S.lastname
)
THEN UPDATE SET 
    T.is_current = FALSE,
    T.end_date_ts = current_timestamp()
WHEN NOT MATCHED THEN INSERT(
    customername,
    phone,
    addressline1,
    addressline2,
    city,
    state,
    postalcode,
    country,
    territory,
    country_iso,
    dealsize,
    firstname,
    lastname,
    is_current,
    start_date_ts,
    end_date_ts
)
VALUES(
    S.customername,
    S.phone,
    S.addressline1,
    S.addressline2,
    S.city,
    S.state,
    S.postalcode,
    S.country,
    S.territory,
    S.country_iso,
    S.dealsize,
    S.firstname,
    S.lastname,
    TRUE,
    current_timestamp(),
    NULL
)

### **FACT LOAD**

In [0]:
WITH fact_src AS (
    SELECT DISTINCT
        s.ordernumber,
        s.productcode,
        c.customer_id,   
        (YEAR(s.orderdate) * 10000) +
        (MONTH(s.orderdate) * 100) +
        DAY(s.orderdate) AS date_key,
        s.quantityordered,
        s.priceeach,
        s.sales,
        s.status
    FROM retail_databricks_ws.silver.silver_cleaned_tbl s
    LEFT JOIN retail_databricks_ws.gold.customers_dim c
        ON s.customername = c.customername
        AND c.is_current = TRUE
)
MERGE INTO retail_databricks_ws.gold.fact_sales AS T
USING fact_src AS S
ON T.ordernumber = S.ordernumber AND
   T.productcode = S.productcode
WHEN MATCHED AND(
    T.customer_id     != S.customer_id OR
    T.date_key        != S.date_key OR
    T.quantityordered != S.quantityordered OR
    T.priceeach       != S.priceeach OR
    T.sales           != S.sales OR
    T.status          != S.status
)
THEN UPDATE SET
    T.customer_id     = S.customer_id,
    T.date_key        = S.date_key,
    T.quantityordered = S.quantityordered,
    T.priceeach       = S.priceeach,
    T.sales           = S.sales,
    T.status           = S.status
WHEN NOT MATCHED THEN INSERT(
    ordernumber,
    productcode,
    customer_id,
    date_key,
    quantityordered,
    priceeach,
    sales,
    status
)
VALUES(
    S.ordernumber,
    S.productcode,
    S.customer_id,
    S.date_key,
    S.quantityordered,
    S.priceeach,
    S.sales,
    S.status
)
